# CSE5IDP Gartner Project by Agile Avengers

#### Run the below code to install and check YOLO

In [1]:
# Pip install method (recommended)

!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO

from IPython.display import display, Image

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.6/107.7 GB disk)


#### After running the above code once you can run the below code as many times you want using different or same pictures. Just make sure to change the image name and tune the hyper parameters accordingly.

In [6]:
# Main source code

# Importing Libraries
import cv2
import numpy as np
from ultralytics import YOLO
from scipy.spatial import distance
import networkx as nx  # For graph and connected component analysis

# Initialize YOLO model (replace 'yolov8n.pt' with the desired model)
model = YOLO('yolov8n.pt')

# Set the image path (replace 'test/9.jpeg' with your actual image)
image_path = 'test/1.jpeg'

# Perform detection on the image, only detecting class 0 (person)
results = model.predict(source=image_path, conf=0.4, classes=0, save=True)

# Load the image for visualization
image = cv2.imread(image_path)

# Initialize lists to store bounding box centers, coordinates, and areas
bbox_centers = []
bbox_coords = []
bbox_areas = []

# Iterate over each detection result
for result in results:
    boxes = result.boxes.xyxy.cpu().numpy()  # Bounding box coordinates
    confidences = result.boxes.conf.cpu().numpy()  # Confidence scores
    class_ids = result.boxes.cls.cpu().numpy()  # Class ids (people in this case)

    # Draw bounding boxes around detected people
    for i, box in enumerate(boxes):
        x_min, y_min, x_max, y_max = box.astype(int)
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)  # Green bounding box for person
        print(f"Person detected: Bounding Box Coordinates: {box}, Confidence: {confidences[i]}, Class ID: {class_ids[i]}")

        # Compute the center and area of the bounding box
        center_x = (x_min + x_max) // 2
        center_y = (y_min + y_max) // 2
        area = (x_max - x_min) * (y_max - y_min)

        bbox_centers.append([center_x, center_y])
        bbox_coords.append([x_min, y_min, x_max, y_max])
        bbox_areas.append(area)  # Keep track of bounding box area

# Convert bbox_centers to a numpy array
bbox_centers = np.array(bbox_centers)

# Parameters to tune
min_distance = 25      # Minimum distance between people to be considered in the same group
max_distance = 150     # Maximum distance threshold for grouping
area_threshold_ratio = 0.5  # Bounding box area ratio threshold for distance-based grouping

# Step 1: Graph Construction
# Create a graph where each node is a person, and edges exist between people within the distance threshold
G = nx.Graph()
for i in range(len(bbox_centers)):
    G.add_node(i)

# Add edges between people who are close enough (within social distance constraints) and similar in size
for i in range(len(bbox_centers)):
    for j in range(i + 1, len(bbox_centers)):
        # Compute centroid distance
        dist = distance.euclidean(bbox_centers[i], bbox_centers[j])

        # Calculate area ratio
        area_ratio = min(bbox_areas[i], bbox_areas[j]) / max(bbox_areas[i], bbox_areas[j])

        # Check if both distance and area ratio meet the thresholds
        if min_distance <= dist <= max_distance and area_ratio >= area_threshold_ratio:
            G.add_edge(i, j)

# Step 2: Find Connected Components (Groups)
connected_components = list(nx.connected_components(G))

# Step 3: Filter out single-person groups
group_components = [group for group in connected_components if len(group) > 1]

# Step 4: Visualize groups (Draw bounding boxes around groups)
for group in group_components:
    group_indices = list(group)

    # Get bounding box coordinates for all people in the group
    group_x_min = min(bbox_coords[idx][0] for idx in group_indices)
    group_y_min = min(bbox_coords[idx][1] for idx in group_indices)
    group_x_max = max(bbox_coords[idx][2] for idx in group_indices)
    group_y_max = max(bbox_coords[idx][3] for idx in group_indices)

    # Draw a larger bounding box around the group
    cv2.rectangle(image, (group_x_min, group_y_min), (group_x_max, group_y_max), (255, 0, 0), 3)  # Blue bounding box for group

# Density-based methods: Create a density heatmap based on person locations
def create_density_heatmap(bbox_centers, image_shape, sigma=15):
    heatmap = np.zeros((image_shape[0], image_shape[1]), dtype=np.float32)

    for center in bbox_centers:
        x, y = int(center[0]), int(center[1])
        heatmap[y, x] += 1  # Increment heatmap at each person's location

    heatmap = cv2.GaussianBlur(heatmap, (0, 0), sigma)
    heatmap = cv2.normalize(heatmap, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    heatmap_colored = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    return heatmap_colored

# Generate heatmap
image_shape = image.shape[:2]
heatmap = create_density_heatmap(bbox_centers, image_shape)

# Save and display the final output with both individual and group bounding boxes
cv2.imwrite('group_detection_output.jpg', image)
cv2.imwrite('density_heatmap.jpg', heatmap)



image 1/1 /content/test/1.jpeg: 288x640 8 persons, 117.0ms
Speed: 2.0ms preprocess, 117.0ms inference, 1.3ms postprocess per image at shape (1, 3, 288, 640)
Results saved to runs/detect/predict4


Person detected: Bounding Box Coordinates: [     854.24      131.08      992.27       472.2], Confidence: 0.8006943464279175, Class ID: 0.0
Person detected: Bounding Box Coordinates: [     495.21      144.98      624.65       454.4], Confidence: 0.7956050038337708, Class ID: 0.0
Person detected: Bounding Box Coordinates: [     745.91      126.59       885.6      484.88], Confidence: 0.743084728717804, Class ID: 0.0
Person detected: Bounding Box Coordinates: [     346.73      163.74      396.99      360.07], Confidence: 0.7022331357002258, Class ID: 0.0
Person detected: Bounding Box Coordinates: [      211.4      114.76      357.62      403.85], Confidence: 0.5746599435806274, Class ID: 0.0
Person detected: Bounding Box Coordinates: [     184.18      194.09      320.98      430.19], Confidence: 0.5475128889083862, Class ID: 0.0
Person detected: Bounding Box Coordinates: [     213.11      115.09      356.87      296.37], Confidence: 0.4261578619480133, Class ID: 0.0
Person detected: Boun

True